In [1]:
import numpy as np
import pandas as pd
import geopandas as gp
import os
import subprocess
import math
import shapely as shp
from datetime import datetime
from fiona.crs import from_epsg
import matplotlib.pylab as plt
import re
from pandas.tseries.holiday import get_calendar, HolidayCalendarFactory, GoodFriday
from datetime import datetime
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score as rs
from sklearn import svm, datasets

from itertools import compress

np.random.seed(222)

%matplotlib inline
%pylab inline

/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


### Notes: current status is informative, but program is also informative

### perhaps add a feature that is the number of occupied properties within a buffer of every property - alice will send over parcel_ID lat/long data

## Notes 2-1-17
1. add MCM condition to dummy variables ###CHECK
2. add any houses that score very high on the occupancy model as homes that should not be demolished ###CHECK
3. update DFD fire list
4. figure out regularization techniques for logit "regression" model ###check
5. run accuracy scores / 'mse' ###check

## Notes 2-13-17

1. add feature that is the number of vacant homes in the census block that a given house belongs to
2. alternative  - add feature that is the # of vacant homes in a given buffer from a property
3. start writing up documentation 


## Notes 2-15-17

1. apply model to full dataset ###CHECK
2. add continuous probabilities to model outputs
3. write up timesheet by next wednesday
4. final paper written by March 2nd?

## Notes 2-21-17
1. build SVM model?
2. I need lat / lon of parcels - Parcel2016.gdb! in the data folder!
3. finalize model, export to .csv

## Notes 2-22-17
1. tune hyperparameters of randomtree
2. build SVM
3. finalize model, export to .csv

In [2]:
wd = os.getcwd()
os.chdir('../data')
os.getcwd()

'/Users/geoffperrin/Desktop/DLBA/demo_project/data'

In [3]:
#read in detroit feature dataset
city_df = pd.read_csv('CitywideIndicators_20170221.csv')

/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## build labeled training dataset

In [4]:
city_df.columns

Index([u'Account ID', u'Parcel ID', u'Account Name', u'Property Ownership',
       u'Property Class', u'Program', u'Current Status',
       u'Non-City Demo Program', u'Current Status Date', u'Neighborhood',
       u'Neighborhood Sales Median', u'DFD Fire: Occurrence Date',
       u'USPS: Vacancy Indicated', u'USPS: Last Update',
       u'Occupancy Model: Probability Pct', u'Occupancy Model: Last Update',
       u'DNG Number', u'Foreclosure Year', u'WCT Foreclosure Status',
       u'MCM Blext Created At', u'MCM Structure', u'MCM Occupancy', u'MCM Use',
       u'MCM Units', u'MCM Residence Type', u'MCM Commercial Use',
       u'MCM Industrial Use', u'MCM Institutional Use', u'MCM Public Use',
       u'MCM Condition', u'MCM Fire Damage', u'MCM Level of Fire Damage',
       u'MCM Boarding Needed', u'MCM Trespass', u'MCM Vacant Lot',
       u'MCM Maintained', u'MCM Dumping'],
      dtype='object')

In [5]:
#remove lots
dlba_df = city_df[city_df['Property Class'] != 'Residential-Lot']

#only keep residential lots
dlba_df = dlba_df[dlba_df['Property Class'] == 'Residential-Structure']


dlba_df_small = dlba_df.rename(columns=lambda x: x.replace(' ', '_'))
dlba_df_small.head()

,Account_ID,Parcel_ID,Account_Name,Property_Ownership,Property_Class,Program,Current_Status,Non-City_Demo_Program,Current_Status_Date,Neighborhood,...,MCM_Institutional_Use,MCM_Public_Use,MCM_Condition,MCM_Fire_Damage,MCM_Level_of_Fire_Damage,MCM_Boarding_Needed,MCM_Trespass,MCM_Vacant_Lot,MCM_Maintained,MCM_Dumping
1,001j000000Idvvm,22027073.,19759 Littlefield,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Greenwich,...,NaN,NaN,Good,NaN,NaN,No,NaN,NaN,NaN,No
2,001j000000Idxml,22054984.,15454 Prevost,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Crary/St Marys,...,NaN,NaN,Good,NaN,NaN,No,NaN,NaN,NaN,No
3,001j000000Idzep,21077398.,6150 University Pl,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Cornerstone Village,...,NaN,NaN,Good,No,NaN,No,NaN,NaN,NaN,No
4,001j000000Ie1Xa,22059049.,15805 St Marys,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Crary/St Marys,...,NaN,NaN,Good,NaN,NaN,Yes,NaN,NaN,NaN,No
5,001j000000Ie3Rg,22062755.,14740 Woodmont,Non-DLBA Owned,Residential-Structure,NaN,NaN,NaN,NaN,Grand River-St Marys,...,NaN,NaN,Good,NaN,NaN,No,NaN,NaN,NaN,No


## need to fix nans in neighborhood sales median
### all the neighborhoods should be in there, but some rows have NaNs!

In [6]:
dlba_df_neighborhood = dlba_df_small[['Neighborhood', 'Neighborhood_Sales_Median']]
dlba_df_neighborhood.groupby(['Neighborhood'], as_index=False).mean().head()

,Neighborhood,Neighborhood_Sales_Median
0,Airport Sub,0.57
1,Arden Park,NaN
2,Aviation Sub,6.24
3,Bagley,11.68
4,Barton-McFarland,1.24


In [7]:
#read in detroit parcel distance to school df
school_df = pd.read_csv('DistanceToSchool_20170123.csv')
school_df_small = school_df[["PARCELNO", "DistanceToSchool_FT"]]
school_df_small.columns = ['Parcel_ID', 'dist_to_school']
school_df_small.head()

,Parcel_ID,dist_to_school
0,16006256.,2196.731029
1,04000103-4,2375.448135
2,04003341.,2442.534185
3,04003339-40,2510.715691
4,04000119.,2341.253487


In [8]:
#merge distance to school to city feature df
dlba_school_df = dlba_df_small.merge(school_df_small, on='Parcel_ID')
len(dlba_school_df)

233839

In [9]:
#code dummy variables

#mcm condition dummies
dlba_school_df['mcm_good_dummy'] = 0
dlba_school_df['mcm_good_dummy'] = [1 if i=='Good' else 0 for i in dlba_school_df['MCM_Condition']]

dlba_school_df['mcm_fair_dummy'] = 0
dlba_school_df['mcm_fair_dummy'] = [1 if i=='Fair' else 0 for i in dlba_school_df['MCM_Condition']]

dlba_school_df['mcm_poor_dummy'] = 0
dlba_school_df['mcm_poor_dummy'] = [1 if (i=='Poor' or i=='poor, fair') else 0 for i in dlba_school_df['MCM_Condition']]

dlba_school_df['mcm_demo_dummy'] = 0
dlba_school_df['mcm_demo_dummy'] = [1 if (i=='Suggest Demolition' or i=='suggest demolition, poor') else 0 for i in dlba_school_df['MCM_Condition']]

dlba_school_df['mcm_good_dummy'] = 0
dlba_school_df['mcm_good_dummy'] = [1 if i=='Good' else 0 for i in dlba_school_df['MCM_Condition']]

dlba_school_df['dng_dummy'] = 0
dlba_school_df['dng_dummy'] = [0 if pd.isnull(i) else 1 for i in dlba_school_df['DNG_Number']]

dlba_school_df['fire_dummy'] = 0
dlba_school_df['fire_dummy'] = [0 if pd.isnull(i) else 1 for i in dlba_school_df['DFD_Fire:_Occurrence_Date']]

dlba_school_df['usps_vacant_dummy'] = 0
dlba_school_df['usps_vacant_dummy'] = [0 if pd.isnull(i) else 1 for i in dlba_school_df['USPS:_Vacancy_Indicated']]

dlba_school_df['foreclosed_dummy'] = 0
dlba_school_df['foreclosed_dummy'] = [0 if pd.isnull(i) else 1 for i in dlba_school_df['Foreclosure_Year']]

#change to current date
dlba_school_df['foreclosure_lag'] = 0
dlba_school_df['foreclosure_lag'] = [0 if pd.isnull(i) else (2017 - i) for i in dlba_school_df['Foreclosure_Year']]

dlba_school_df['boarding_needed_dummy'] = 0
dlba_school_df['boarding_needed_dummy'] = [0 if (pd.isnull(i) or i == 'No') else 1 for i in dlba_school_df['MCM_Boarding_Needed']]

dlba_school_df['dumping_dummy'] = 0
dlba_school_df['dumping_dummy'] = [0 if (pd.isnull(i) or i == 'No') else 1 for i in dlba_school_df['MCM_Dumping']]

#replace NaNs with zeros
dlba_school_df['Occupancy_Model:_Probability_Pct'] = [0 if pd.isnull(i) else i for i in dlba_school_df['Occupancy_Model:_Probability_Pct']]

#remove NaNs for Neighborhood_Sales_Median
#in final roll out I need to improve this!!!
dlba_school_df = dlba_school_df[~pd.isnull(dlba_school_df['Neighborhood_Sales_Median'])]




In [10]:
# create a full copy of the data in order to apply model to it at the end
dlba_school_df_full = dlba_school_df.copy()

dlba_school_df_full_small = dlba_school_df_full.drop(['Account_ID', 'Property_Ownership', 'Property_Class', 'Program', \
                                            'Current_Status', 'Current_Status_Date', 'Neighborhood', 'DNG_Number', \
                                            'WCT_Foreclosure_Status', \
                                            'MCM_Blext_Created_At', 'MCM_Structure', 'MCM_Occupancy',  'MCM_Use',\
                                           'MCM_Units', 'MCM_Residence_Type', 'MCM_Commercial_Use', 'MCM_Industrial_Use',\
                                           'MCM_Institutional_Use', 'MCM_Public_Use', 'MCM_Level_of_Fire_Damage', \
                                            'MCM_Trespass', 'MCM_Vacant_Lot', 'MCM_Maintained', 'DFD_Fire:_Occurrence_Date',\
                                           'USPS:_Vacancy_Indicated', 'Foreclosure_Year', 'MCM_Fire_Damage',\
                                           'USPS:_Last_Update', 'Occupancy_Model:_Last_Update', 'MCM_Boarding_Needed',\
                                           'MCM_Dumping', 'MCM_Condition'], 1)

dlba_school_df_full_small = dlba_school_df_full_small.rename(columns={'Occupancy_Model:_Probability_Pct': 'occupancy_model_pct'})
dlba_school_df_full_small = dlba_school_df_full_small[['Parcel_ID', 'Account_Name', 'Neighborhood_Sales_Median',\
                                            'occupancy_model_pct', 'dist_to_school',\
                                            'fire_dummy', 'usps_vacant_dummy', 'foreclosed_dummy',\
                                            'foreclosure_lag', 'boarding_needed_dummy', 'dumping_dummy', 'dng_dummy',\
                                            'mcm_good_dummy', 'mcm_fair_dummy', 'mcm_demo_dummy']]


In [11]:
dlba_school_df[dlba_school_df['Current_Status']=="Dismissed"]

,Account_ID,Parcel_ID,Account_Name,Property_Ownership,Property_Class,Program,Current_Status,Non-City_Demo_Program,Current_Status_Date,Neighborhood,...,mcm_fair_dummy,mcm_poor_dummy,mcm_demo_dummy,dng_dummy,fire_dummy,usps_vacant_dummy,foreclosed_dummy,foreclosure_lag,boarding_needed_dummy,dumping_dummy
16,001j000000IeT1K,20003215.,8840 Falcon,Former Property of Interest,Residential-Structure,NAP,Dismissed,NaN,4/19/2016,Springwells,...,0,0,0,0,0,1,0,0.0,0,0
106,001j000000IeKvI,22114623.,7474 Dolphin,Former Property of Interest,Residential-Structure,NAP,Dismissed,NaN,7/6/2016,Far West Detroit,...,0,0,0,0,0,1,1,6.0,1,0
137,001j000000IeHC9,22116615.,7501 Lamphere,Former Property of Interest,Residential-Structure,NAP,Dismissed,NaN,10/27/2016,Far West Detroit,...,0,0,0,0,0,1,1,6.0,0,0
149,001j000000Idvvv,16034897.,19174 Wisconsin,Former Property of Interest,Residential-Structure,NAP,Dismissed,NaN,6/25/2015,Bagley,...,0,0,0,0,0,1,0,0.0,0,0
483,001j000000IdvwF,16031924.,18104 Northlawn,Former Property of Interest,Residential-Structure,NAP,Dismissed,NaN,9/3/2015,Bagley,...,1,0,0,0,0,1,0,0.0,0,1
862,001j000000IeWpX,14005211.,4020 Buena Vista,Former Property of Interest,Residential-Structure,NAP,Dismissed,NaN,10/26/2016,Russell Woods,...,0,0,0,0,0,1,1,4.0,0,0
1468,001j000000Ie1Yz,16032870.,16550 Cherrylawn,Former Property of Interest,Residential-Structure,NAP,Dismissed,NaN,6/22/2015,Fitzgerald/Marygrove,...,0,0,0,0,0,1,0,0.0,0,0
1591,001j000000IeMtO,22091669.,6201 Minock,Former Property of Interest,Residential-Structure,NAP,Dismissed,NaN,2/13/2015,Warrendale,...,0,0,0,0,1,1,0,0.0,0,0
1634,001j000000Ie1ZA,22059041.,15871 St Marys,Former Property of Interest,Residential-Structure,NAP,Dismissed,NaN,5/18/2016,Crary/St Marys,...,0,0,0,0,0,1,0,0.0,1,1
1919,001j000000IeHDs,22116619.,7467 Lamphere,Former Property of Interest,Residential-Structure,NAP,Dismissed,NaN,5/13/2016,Far West Detroit,...,0,0,0,1,0,1,1,5.0,1,0


In [12]:
#need to only apply demo decision to above occupancy threshold & dlba owned properties
#only keep DLBA Owned properties for the training data OR with a very high occupancy score

no_demo_occupancy_thresh = 95.0
dlba_school_df = dlba_school_df[(dlba_school_df['Property_Ownership'] == 'DLBA Owned') | \
                  (dlba_school_df['Occupancy_Model:_Probability_Pct'] > no_demo_occupancy_thresh)]

#code up demo decision yes / no
demo_decision_dict = {'Demo Awaiting Funding': 1,
        'Demo Pipeline': 1,
        'Demo Contracted': 1,
        'Demo Review': 1,
        'Demolished': 999,
        'Construction Complete': 0,
        'Occupied': 0,
        'Sold': 0,
        'Institutional Partner Coordination': 999,
        'For Sale': 0,
        'Sales Review': 0,
        'Sales Pipeline': 0,
        'Vacant Land': 999,
        'Sales Hold': 0,
        'Under Construction': 999,
        'Lease/ROE/TLA': 999,
        'Warehouse': 999,
        'NPOC Isolation - Do not move!': 999,
        'NSP Quarantine': 999,
        'Reconveyance Complete': 999,
        'Title Problem Identified': 999,
        'DJ Quarantine': 999,
        'PDD Quarantine': 999,
        'Intake Review': 999}

#if occupancy score > threshold, then no demo
dlba_school_df['Current_Status']\
[dlba_school_df['Occupancy_Model:_Probability_Pct'] > no_demo_occupancy_thresh] = 'Occupied'

#if MLB demo, then yes demo
dlba_school_df['Current_Status']\
[dlba_school_df['Non-City_Demo_Program'] =='MLB Demo'] = 'Demo Pipeline'

dlba_school_df['demo_decision'] = 0
dlba_school_df['demo_decision'] = [demo_decision_dict[i] for i in dlba_school_df["Current_Status"]]



dlba_school_df = dlba_school_df[dlba_school_df['demo_decision'] != 999]

dlba_school_df.head()

/Users/geoffperrin/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Account_ID,Parcel_ID,Account_Name,Property_Ownership,Property_Class,Program,Current_Status,Non-City_Demo_Program,Current_Status_Date,Neighborhood,...,mcm_poor_dummy,mcm_demo_dummy,dng_dummy,fire_dummy,usps_vacant_dummy,foreclosed_dummy,foreclosure_lag,boarding_needed_dummy,dumping_dummy,demo_decision
0,001j000000Idvvm,22027073.,19759 Littlefield,Non-DLBA Owned,Residential-Structure,NaN,Occupied,NaN,NaN,Greenwich,...,0,0,0,0,1,0,0.0,0,0,0
1,001j000000Idxml,22054984.,15454 Prevost,Non-DLBA Owned,Residential-Structure,NaN,Occupied,NaN,NaN,Crary/St Marys,...,0,0,0,0,1,0,0.0,0,0,0
5,001j000000Ie5Ko,16027453.,16128 Turner,Non-DLBA Owned,Residential-Structure,NaN,Occupied,NaN,NaN,Fitzgerald/Marygrove,...,0,0,0,0,1,0,0.0,0,0,0
6,001j000000Ie7Ej,12006273.,16645 Lawton,Non-DLBA Owned,Residential-Structure,NaN,Occupied,NaN,NaN,Martin Park,...,0,0,0,0,1,0,0.0,0,0,0
8,001j000000IeBUo,10006510.,1483 16th St,Non-DLBA Owned,Residential-Structure,NaN,Occupied,NaN,NaN,Hubbard Richard,...,0,0,0,0,1,0,0.0,0,0,0


In [13]:
# only keep DLBA Owned properties for the training data OR with a very high occupancy score: we only want
# labelled data in the training dataset

dlba_school_df = dlba_school_df[(dlba_school_df['Property_Ownership'] == 'DLBA Owned') | \
                  (dlba_school_df['Occupancy_Model:_Probability_Pct'] > no_demo_occupancy_thresh)]

dlba_school_df_small = dlba_school_df.drop(['Account_ID', 'Property_Ownership', 'Property_Class', 'Program', \
                                            'Current_Status', 'Current_Status_Date', 'Neighborhood', 'DNG_Number', \
                                            'WCT_Foreclosure_Status', \
                                            'MCM_Blext_Created_At', 'MCM_Structure', 'MCM_Occupancy',  'MCM_Use',\
                                           'MCM_Units', 'MCM_Residence_Type', 'MCM_Commercial_Use', 'MCM_Industrial_Use',\
                                           'MCM_Institutional_Use', 'MCM_Public_Use', 'MCM_Level_of_Fire_Damage', \
                                            'MCM_Trespass', 'MCM_Vacant_Lot', 'MCM_Maintained', 'DFD_Fire:_Occurrence_Date',\
                                           'USPS:_Vacancy_Indicated', 'Foreclosure_Year', 'MCM_Fire_Damage',\
                                           'USPS:_Last_Update', 'Occupancy_Model:_Last_Update', 'MCM_Boarding_Needed',\
                                           'MCM_Dumping', 'MCM_Condition'], 1)

dlba_school_df_small = dlba_school_df_small.rename(columns={'Occupancy_Model:_Probability_Pct': 'occupancy_model_pct'})
dlba_school_df_small = dlba_school_df_small[['Parcel_ID', 'Account_Name', 'demo_decision', 'Neighborhood_Sales_Median',\
                                            'occupancy_model_pct', 'dist_to_school',\
                                            'fire_dummy', 'usps_vacant_dummy', 'foreclosed_dummy',\
                                            'foreclosure_lag', 'boarding_needed_dummy', 'dumping_dummy', 'dng_dummy',\
                                            'mcm_good_dummy', 'mcm_fair_dummy', 'mcm_demo_dummy']]

dlba_school_df_small.head(5)

,Parcel_ID,Account_Name,demo_decision,Neighborhood_Sales_Median,occupancy_model_pct,dist_to_school,fire_dummy,usps_vacant_dummy,foreclosed_dummy,foreclosure_lag,boarding_needed_dummy,dumping_dummy,dng_dummy,mcm_good_dummy,mcm_fair_dummy,mcm_demo_dummy
0,22027073.,19759 Littlefield,0,4.16,99.66,1390.037738,0,1,0,0.0,0,0,0,1,0,0
1,22054984.,15454 Prevost,0,8.49,99.65,2291.720306,0,1,0,0.0,0,0,0,1,0,0
5,16027453.,16128 Turner,0,3.00,100.00,979.713445,0,1,0,0.0,0,0,0,1,0,0
6,12006273.,16645 Lawton,0,1.42,99.48,852.641974,0,1,0,0.0,0,0,0,1,0,0
8,10006510.,1483 16th St,0,9.11,100.00,2290.365887,0,1,0,0.0,0,0,0,1,0,0


## Build Logit Classification Model

In [14]:
y = dlba_school_df_small.demo_decision
X = dlba_school_df_small.iloc[:,3:]
'''
let's try this without the crossvalidation loop first


for i in range(10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = i)
    
    #learn logistic regression over the training sample
    logit = LogisticRegression(C=1)
    logit.fit(X_train, y_train)
    # C here controls the regularization strength--which will be discussed in Bayesian session
    # For now just keep in mind that setting a high value will make the regularization ineffective

    #apply to the validation sample
    labelPred = logit.predict(X_test)
    #labelPred = logPred > 0.5
    labelActual = y_test
    
'''    


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 123)
    
#learn logistic regression over the training sample
logit = LogisticRegression(C=1)
logit.fit(X_train, y_train)
# C here controls the regularization strength--which will be discussed in Bayesian session
# For now just keep in mind that setting a high value will make the regularization ineffective

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

### logit model - Accuracy

In [15]:
labelActual = y_test


In [16]:
#apply to the validation sample
labelPred = logit.predict(X_test)
X_test_copy = X_test.copy()
X_test_copy['logit_predict'] = logit.predict(X_test)
sum(labelPred)

3251

In [17]:
#compute TP,FP,FN,TN for the above classification
TP = sum((labelPred==1)&(labelActual==1))
FP = sum((labelPred==1)&(labelActual==0))
FN = sum((labelPred==0)&(labelActual==1))
TN = sum((labelPred==0)&(labelActual==0))

#compute precision, accuracy and recall
Acc = 100.0 * (TP+TN)/(TP+TN+FP+FN)
Rec = 100.0 * TP/(TP+FN)
Prec= 100.0 * TP/(TP+FP)
print('Precision = %.2f%%\nAccuracy = %.2f%%\nRecall = %.2f%%'%(Prec,Acc,Rec))

Precision = 80.74%
Accuracy = 96.73%
Recall = 76.89%


#### look at auc

In [ ]:
res=[]
for i in range(1,20):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = i)
    pred=logit.predict_proba(X_test)[:,1]
    res.append(rs(np.array(y_test.apply(int)),pred))
    #print rs(np.array(y_test.apply(int)),pred)
print np.mean(res)

0.989015253436


## Build Random Forest Classification Model

In [ ]:
r_tree = RFC()
param_grid = {'max_depth': [3, 5, 7, 9], 'n_estimators': np.arange(1,502,100)}
r_tree_tuned = GridSearchCV(r_tree, param_grid)
#r_tree_optimum = RFC(r_tree_tuned.best_estimator_)
r_tree_tuned.fit(X_train, y_train)

In [ ]:
feature_importance = r_tree_tuned.best_estimator_.fit(X_train, y_train).feature_importances_

In [ ]:
Feature_importance_df=pd.DataFrame([list(X_train.columns),list(feature_importance)]).T
Feature_importance_df.columns=["variables","importance"]
Feature_importance_df.sort_values(by="importance",ascending=False)

### random forest model - Accuracy

In [ ]:
#apply to the validation sample
labelPred = r_tree_tuned.predict(X_test)
labelActual = y_test
X_test_copy['r_tree_predict'] = r_tree_tuned.predict(X_test)
X_test_copy['labelActual'] = y_test
sum(labelPred)

#### look at precision / accuracy / recall

In [ ]:
#compute TP,FP,FN,TN for the above classification
TP = sum((labelPred==1)&(labelActual==1))
FP = sum((labelPred==1)&(labelActual==0))
FN = sum((labelPred==0)&(labelActual==1))
TN = sum((labelPred==0)&(labelActual==0))

#compute precision, accuracy and recall
Prec= 100.0 * TP/(TP+FP)
Acc = 100.0 * (TP+TN)/(TP+TN+FP+FN)
Rec = 100.0 * TP/(TP+FN)
print('Precision = %.2f%%\nAccuracy = %.2f%%\nRecall = %.2f%%'%(Prec,Acc,Rec))

#### look at auc

In [ ]:
res=[]
for i in range(1,20):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = i)
    pred=r_tree_tuned.predict_proba(X_test)[:,1]
    res.append(rs(np.array(y_test.apply(int)),pred))
print np.mean(res)

In [ ]:
### merge on address
dlba_school_df_address = dlba_school_df_full[['Account_Name']]
test_data_full = X_test_copy.merge(dlba_school_df_address, how='left', left_index=True, right_index=True) 

In [ ]:
test_data_full[((test_data_full.r_tree_predict==1) & (test_data_full.labelActual==0)) | ((test_data_full.r_tree_predict==0) & (test_data_full.labelActual==1))].tail(10)

## SVM Model 
### - not implimenting this, takes way too long to fit, takes many, MANY hours to actually tune hyperparameters

In [ ]:
#parameters = {'kernel':('linear', 'rbf'), 'C':[1, 5]}
#svr = svm.SVC()
#clf = GridSearchCV(svr, parameters)
#svr_optimum = svr.fit(X_train, y_train)

In [ ]:
#parameters = {'kernel':('linear', 'rbf'), 'C':[1, 500, 1000, 10000]}
#svr = svm.SVC()
#clf = GridSearchCV(svr, parameters)
#svr_optimum = clf.fit(X_train, y_train)

### SVM - Accuracy

In [ ]:
#apply to the validation sample
#labelPred = svr_optimum.predict(X_test)
#labelActual = y_test
#X_test_copy['SVM_predict'] = svr_optimum.predict(X_test)

#### look at precision / accuracy / recall

In [ ]:
'''
#compute TP,FP,FN,TN for the above classification
TP = sum((labelPred==1)&(labelActual==1))
FP = sum((labelPred==1)&(labelActual==0))
FN = sum((labelPred==0)&(labelActual==1))
TN = sum((labelPred==0)&(labelActual==0))

#compute precision, accuracy and recall
Prec= 100.0 * TP/(TP+FP)
Acc = 100.0 * (TP+TN)/(TP+TN+FP+FN)
Rec = 100.0 * TP/(TP+FN)
print('Precision = %.2f%%\nAccuracy = %.2f%%\nRecall = %.2f%%'%(Prec,Acc,Rec))
'''

#### look at auc

In [ ]:
'''
res=[]
for i in range(1,20):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = i)
    pred=svr_optimum.predict(X_test)
    res.append(rs(np.array(y_test.apply(int)),pred))
    #print rs(np.array(y_test.apply(int)),pred)
print np.mean(res)
'''

## Apply models to full dataset

In [ ]:
# apply to full dataset
dlba_school_df_full_small_copy = dlba_school_df_full_small.copy()
dlba_school_df_full_small_X = dlba_school_df_full_small_copy.iloc[:,2:]
dlba_school_df_full_small['r_tree_predict'] = r_tree.predict(dlba_school_df_full_small_X)
dlba_school_df_full_small['r_tree_predict_continuous'] = r_tree.predict_proba(dlba_school_df_full_small_X)[:,1]

In [ ]:
### find addresses of interest
# 456 harper
dlba_school_df_full_small[dlba_school_df_full_small.Account_Name.str.contains("Parkland")]

## Export final dataset as csv

In [ ]:
dlba_school_df_full_small.to_csv("/Users/geoffperrin/Desktop/DLBA/demo_project/output/final_demo_prediction_022217.csv")